In [7]:
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import time
from datetime import datetime, timedelta
from scipy.signal import argrelextrema
import plotly.express as px
import plotly.io as pio

# Set default template for better appearance
pio.templates.default = "plotly_white"

def get_sp500_tickers():
    """
    Retrieves current S&P 500 components from Wikipedia
    
    Returns:
        list: List of S&P 500 ticker symbols
    """
    try:
        # URL for the S&P 500 components table on Wikipedia
        url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
        
        # Read the tables from the Wikipedia page
        tables = pd.read_html(url)
        
        # The first table contains the S&P 500 components
        sp500_table = tables[0]
        
        # Extract the 'Symbol' column as a list
        tickers = sp500_table['Symbol'].tolist()
        
        # Clean up the tickers (replace dots with hyphens for BRK.B, etc.)
        tickers = [ticker.replace('.', '-') for ticker in tickers]
        
        return tickers
    
    except Exception as e:
        print(f"Error retrieving S&P 500 tickers: {e}")
        return []

def get_daily_data(tickers, start_date='2020-01-01', end_date=None, batch_size=520):
    """
    Retrieves daily price data for a list of tickers in batches to avoid API limitations
    
    Args:
        tickers (list): List of ticker symbols
        start_date (str): Start date for data retrieval in YYYY-MM-DD format
        end_date (str): End date for data retrieval in YYYY-MM-DD format
        batch_size (int): Number of tickers to download at once
    
    Returns:
        pd.DataFrame: DataFrame with daily closing prices for each ticker
    """
    if end_date is None:
        end_date = datetime.today().strftime('%Y-%m-%d')
    
    all_data = pd.DataFrame()
    
    # Process tickers in batches to avoid API limitations
    for i in range(0, len(tickers), batch_size):
        batch_tickers = tickers[i:i+batch_size]
        print(f"Downloading data for tickers {i+1} to {min(i+batch_size, len(tickers))}...")
        
        try:
            # Download daily data for the batch
            batch_data = yf.download(batch_tickers, start=start_date, end=end_date, progress=False)
            
            # If we have more than one ticker, we'll have a MultiIndex DataFrame
            if len(batch_tickers) > 1:
                batch_close = batch_data['Close']
            else:
                # For a single ticker, we need to handle differently
                batch_close = batch_data['Close'].to_frame(name=batch_tickers[0])
            
            # For the first batch, set this as our dataframe
            if all_data.empty:
                all_data = batch_close
            else:
                # For subsequent batches, join with existing data
                all_data = all_data.join(batch_close, how='outer')
            
            # Add a small delay to avoid hitting API limits
            time.sleep(1)
            
        except Exception as e:
            print(f"Error downloading data for batch starting at index {i}: {e}")
    
    return all_data

def get_spy_data(start_date='2020-01-01', end_date=None):
    """
    Retrieves SPY ETF data for comparison with A/D line
    
    Args:
        start_date (str): Start date for data retrieval in YYYY-MM-DD format
        end_date (str): End date for data retrieval in YYYY-MM-DD format
    
    Returns:
        pd.DataFrame: DataFrame with SPY daily data with proper column structure
    """
    if end_date is None:
        end_date = datetime.today().strftime('%Y-%m-%d')
        
    try:
        spy_data = yf.download('SPY', start=start_date, end=end_date, progress=False)
        
        # Fix multi-index columns if present
        if len(spy_data.columns.names) > 1:
            spy_data.columns = spy_data.columns.droplevel(1)
            
        print(f"SPY data columns after fixing: {spy_data.columns.tolist()}")
        print(f"First few rows of SPY Close data: {spy_data['Close'].head()}")
        
        return spy_data
    except Exception as e:
        print(f"Error downloading SPY data: {e}")
        return pd.DataFrame()

def calculate_daily_advances_declines(daily_data):
    """
    Calculate the number of advancing and declining stocks each day
    
    Args:
        daily_data (pd.DataFrame): DataFrame with daily closing prices for each ticker
    
    Returns:
        pd.DataFrame: DataFrame with daily counts of advances, declines, and unchanged
    """
    # Calculate daily price changes
    daily_changes = daily_data.pct_change()
    
    # Count advances, declines, and unchanged for each day
    advances = (daily_changes > 0).sum(axis=1)
    declines = (daily_changes < 0).sum(axis=1)
    unchanged = (daily_changes == 0).sum(axis=1)
    
    # Create DataFrame with results
    breadth_data = pd.DataFrame({
        'Advances': advances,
        'Declines': declines,
        'Unchanged': unchanged,
        'Total': advances + declines + unchanged,
        'Advance_Decline_Diff': advances - declines
    })
    
    return breadth_data

def calculate_ad_line(breadth_data, initial_value=0):
    """
    Calculate the Advance/Decline line
    
    Args:
        breadth_data (pd.DataFrame): DataFrame with daily advances and declines
        initial_value (int): Starting value for the A/D line
    
    Returns:
        pd.DataFrame: DataFrame with A/D line and technical indicators
    """
    # Create a copy of the input data
    ad_data = breadth_data.copy()
    
    # Calculate the A/D line (cumulative sum of daily A/D differences)
    ad_data['AD_Line'] = initial_value + ad_data['Advance_Decline_Diff'].cumsum()
    
    # Calculate percentage of advancing stocks
    ad_data['Pct_Advancing'] = (ad_data['Advances'] / ad_data['Total'] * 100).round(2)
    
    # Calculate moving averages
    ad_data['AD_Line_SMA50'] = ad_data['AD_Line'].rolling(window=50).mean()
    ad_data['AD_Line_SMA200'] = ad_data['AD_Line'].rolling(window=200).mean()
    
    # Calculate Rate of Change (ROC) - 20-day
    ad_data['AD_Line_ROC20'] = ad_data['AD_Line'].pct_change(periods=20) * 100
    
    return ad_data

def identify_local_extrema(data, column, order=20):
    """
    Identify local maxima and minima in a time series
    
    Args:
        data (pd.DataFrame): Input DataFrame
        column (str): Column name to analyze
        order (int): How many points on each side to check for extrema
    
    Returns:
        tuple: DataFrames containing local maxima and minima
    """
    # Find local maxima
    max_idx = argrelextrema(data[column].values, np.greater, order=order)[0]
    maxima = data.iloc[max_idx]
    
    # Find local minima
    min_idx = argrelextrema(data[column].values, np.less, order=order)[0]
    minima = data.iloc[min_idx]
    
    return maxima, minima

def detect_divergences(ad_data, spy_data, window=60, threshold=0.7):
    """
    Detect divergences between A/D line and SPY
    
    Args:
        ad_data (pd.DataFrame): A/D line data
        spy_data (pd.DataFrame): SPY price data
        window (int): Window size for divergence detection
        threshold (float): Correlation threshold for divergence
    
    Returns:
        pd.DataFrame: DataFrame with divergence indicators
    """
    # Ensure both datasets have the same index
    ad_data_aligned = ad_data.copy()
    spy_data_aligned = spy_data.copy()
    
    # Get common dates
    common_dates = ad_data_aligned.index.intersection(spy_data_aligned.index)
    
    # Filter data to common dates
    ad_data_aligned = ad_data_aligned.loc[common_dates]
    spy_data_aligned = spy_data_aligned.loc[common_dates]
    
    # Create merged dataframe
    merged_data = pd.DataFrame(index=common_dates)
    merged_data['AD_Line'] = ad_data_aligned['AD_Line']
    merged_data['SPY_Close'] = spy_data_aligned['Close']
    
    # Calculate percentage changes for each
    merged_data['AD_Line_Pct_Change'] = merged_data['AD_Line'].pct_change()
    merged_data['SPY_Pct_Change'] = merged_data['SPY_Close'].pct_change()
    
    # Remove NaN values
    merged_data = merged_data.dropna()
    
    # Rolling correlation between A/D line and SPY
    merged_data['Rolling_Corr'] = merged_data['AD_Line_Pct_Change'].rolling(window=window).corr(
        merged_data['SPY_Pct_Change'])
    
    # Identify potential divergences when correlation falls below threshold
    merged_data['Potential_Divergence'] = (
        merged_data['Rolling_Corr'].abs() < threshold).astype(int)
    
    # Check for periods where A/D line and SPY are moving in opposite directions
    merged_data['Opposite_Direction'] = (
        merged_data['AD_Line_Pct_Change'] * merged_data['SPY_Pct_Change'] < 0).astype(int)
    
    # Combine both conditions for stronger divergence signal
    merged_data['Divergence'] = (
        (merged_data['Potential_Divergence'] == 1) & 
        (merged_data['Opposite_Direction'].rolling(window=5).sum() >= 3)
    ).astype(int)
    
    return merged_data

def fit_trendlines(data, column, max_lines=3):
    """
    Fit simple trendlines to the A/D line data
    
    Args:
        data (pd.DataFrame): Input DataFrame
        column (str): Column name to analyze
        max_lines (int): Maximum number of trendlines to fit
    
    Returns:
        list: List of dictionaries with trendline information
    """
    # Get local extrema
    maxima, minima = identify_local_extrema(data, column)
    
    # For uptrend lines, use minima
    uptrend_points = minima.sort_values(by=data.index.name or 'index', ascending=False).head(max_lines)
    
    # For downtrend lines, use maxima
    downtrend_points = maxima.sort_values(by=data.index.name or 'index', ascending=False).head(max_lines)
    
    trendlines = []
    
    # Process uptrend lines
    for i in range(len(uptrend_points) - 1):
        if len(uptrend_points) < 2:
            continue
            
        point1 = uptrend_points.iloc[i]
        point2 = uptrend_points.iloc[i+1]
        
        # Only process if we have enough distance between points
        if (point1.name - point2.name).days < 20:
            continue
            
        # Linear regression for two points
        x1 = np.datetime64(point2.name)
        y1 = point2[column]
        x2 = np.datetime64(point1.name)
        y2 = point1[column]
        
        # Calculate slope and intercept
        x1_num = x1.astype('datetime64[s]').astype(np.int64)
        x2_num = x2.astype('datetime64[s]').astype(np.int64)
        slope = (y2 - y1) / (x2_num - x1_num)
        intercept = y1 - slope * x1_num
        
        # Add trendline data
        trendlines.append({
            'type': 'uptrend',
            'start_date': point2.name,
            'end_date': point1.name,
            'slope': slope,
            'intercept': intercept,
            'start_value': y1,
            'end_value': y2
        })
    
    # Process downtrend lines (similar to uptrend)
    for i in range(len(downtrend_points) - 1):
        if len(downtrend_points) < 2:
            continue
            
        point1 = downtrend_points.iloc[i]
        point2 = downtrend_points.iloc[i+1]
        
        if (point1.name - point2.name).days < 20:
            continue
            
        x1 = np.datetime64(point2.name)
        y1 = point2[column]
        x2 = np.datetime64(point1.name)
        y2 = point1[column]
        
        x1_num = x1.astype('datetime64[s]').astype(np.int64)
        x2_num = x2.astype('datetime64[s]').astype(np.int64)
        slope = (y2 - y1) / (x2_num - x1_num)
        intercept = y1 - slope * x1_num
        
        trendlines.append({
            'type': 'downtrend',
            'start_date': point2.name,
            'end_date': point1.name,
            'slope': slope,
            'intercept': intercept,
            'start_value': y1,
            'end_value': y2
        })
    
    return trendlines

def create_ad_line_plot(ad_data, spy_data, merged_data, trendlines):
    """
    Create Plotly visualization for A/D line analysis
    
    Args:
        ad_data (pd.DataFrame): A/D line data
        spy_data (pd.DataFrame): SPY price data
        merged_data (pd.DataFrame): Merged data with divergence indicators
        trendlines (list): Trendline information
    
    Returns:
        plotly.graph_objects.Figure: Interactive Plotly figure
    """
    # Create subplot with 3 rows
    fig = make_subplots(rows=3, cols=1, 
                         shared_xaxes=True,
                         vertical_spacing=0.08,
                         row_heights=[0.5, 0.3, 0.2],
                         subplot_titles=("A/D Line with Moving Averages", 
                                         "SPY Price", 
                                         "Breadth Indicators"))
    
    # Plot A/D Line and SMAs
    fig.add_trace(
        go.Scatter(x=ad_data.index, y=ad_data['AD_Line'], 
                   name="A/D Line", 
                   line=dict(color='blue', width=2),
                   hovertemplate='Date: %{x}<br>A/D Line: %{y:.2f}<extra></extra>'),
        row=1, col=1
    )
    
    fig.add_trace(
        go.Scatter(x=ad_data.index, y=ad_data['AD_Line_SMA50'], 
                   name="50-day SMA", 
                   line=dict(color='orange', width=1.5, dash='dash'),
                   hovertemplate='Date: %{x}<br>50-day SMA: %{y:.2f}<extra></extra>'),
        row=1, col=1
    )
    
    fig.add_trace(
        go.Scatter(x=ad_data.index, y=ad_data['AD_Line_SMA200'], 
                   name="200-day SMA", 
                   line=dict(color='red', width=1.5, dash='dash'),
                   hovertemplate='Date: %{x}<br>200-day SMA: %{y:.2f}<extra></extra>'),
        row=1, col=1
    )
    
    # Add trendlines to A/D Line chart
    for tl in trendlines:
        # Generate points for the trendline
        dates = pd.date_range(start=tl['start_date'], end=tl['end_date'])
        dates_num = np.array([pd.Timestamp(d).timestamp() for d in dates])
        values = tl['slope'] * dates_num + tl['intercept']
        
        # Set color based on trend type
        line_color = 'green' if tl['type'] == 'uptrend' else 'red'
        
        fig.add_trace(
            go.Scatter(x=dates, y=values, 
                       mode='lines', 
                       name=f"{tl['type']} {tl['start_date'].strftime('%Y-%m')}",
                       line=dict(color=line_color, width=1, dash='dot'),
                       hoverinfo='skip'),
            row=1, col=1
        )
    
    # Ensure SPY data has the same date range as AD line data
    common_dates = ad_data.index.intersection(spy_data.index)
    spy_aligned = spy_data.loc[common_dates]
    
    # Plot SPY price
    fig.add_trace(
        go.Scatter(x=spy_aligned.index, y=spy_aligned['Close'], 
                   name="SPY", 
                   line=dict(color='green', width=2),
                   hovertemplate='Date: %{x}<br>SPY Price: $%{y:.2f}<extra></extra>'),
        row=2, col=1
    )
    
    # Add 50-day and 200-day SMAs for SPY
    fig.add_trace(
        go.Scatter(x=spy_aligned.index, y=spy_aligned['Close'].rolling(50).mean(), 
                   name="SPY 50-day SMA", 
                   line=dict(color='orange', width=1, dash='dash'),
                   hovertemplate='Date: %{x}<br>SPY 50-day SMA: $%{y:.2f}<extra></extra>'),
        row=2, col=1
    )
    
    fig.add_trace(
        go.Scatter(x=spy_aligned.index, y=spy_aligned['Close'].rolling(200).mean(), 
                   name="SPY 200-day SMA", 
                   line=dict(color='red', width=1, dash='dash'),
                   hovertemplate='Date: %{x}<br>SPY 200-day SMA: $%{y:.2f}<extra></extra>'),
        row=2, col=1
    )
    
    # Plot A/D Line ROC and Percentage Advancing
    fig.add_trace(
        go.Scatter(x=ad_data.index, y=ad_data['AD_Line_ROC20'], 
                   name="A/D Line 20-day ROC", 
                   line=dict(color='purple', width=1.5),
                   hovertemplate='Date: %{x}<br>ROC: %{y:.2f}%<extra></extra>'),
        row=3, col=1
    )
    
    fig.add_trace(
        go.Scatter(x=ad_data.index, y=ad_data['Pct_Advancing'], 
                   name="% Advancing", 
                   line=dict(color='teal', width=1.5),
                   hovertemplate='Date: %{x}<br>% Advancing: %{y:.2f}%<extra></extra>'),
        row=3, col=1
    )
    
    # Add 50% reference line for Percentage Advancing
    fig.add_hline(y=50, line_dash="dash", line_color="gray", row=3, col=1, 
                 annotation_text="50% Advancing", annotation_position="bottom right")
    
    # Add reference line for zero ROC
    fig.add_hline(y=0, line_dash="dash", line_color="black", row=3, col=1)
    
    # Highlight divergences
    if 'Divergence' in merged_data.columns:
        divergence_dates = merged_data[merged_data['Divergence'] == 1].index
        
        for date in divergence_dates:
            # Add vertical lines for divergence points
            fig.add_vline(x=date, line_width=1, line_dash="dash", line_color="red",
                         opacity=0.5, row="all", col=1)
            
            # Add annotations for clarity
            fig.add_annotation(
                x=date,
                y=ad_data.loc[ad_data.index == date, 'AD_Line'].values[0] if date in ad_data.index else 0,
                text="Divergence",
                showarrow=True,
                arrowhead=2,
                arrowsize=1,
                arrowwidth=1,
                arrowcolor="red",
                ax=0,
                ay=-40,
                row=1, col=1
            )
    
    # Update layout
    fig.update_layout(
        title={
            'text': "S&P 500 Advance/Decline Line Analysis",
            'font': {'size': 24, 'color': 'black'}
        },
        height=900,
        legend=dict(
            orientation="h", 
            yanchor="bottom", 
            y=1.02, 
            xanchor="center", 
            x=0.5,
            font=dict(size=10)
        ),
        hovermode="x unified",
        xaxis=dict(
            rangeselector=dict(
                buttons=list([
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=3, label="3m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all")
                ]),
                font=dict(size=10)
            )
        )
    )
    
    # Update y-axis labels
    fig.update_yaxes(title_text="A/D Line Value", title_font=dict(size=12), row=1, col=1)
    fig.update_yaxes(title_text="SPY Price ($)", title_font=dict(size=12), row=2, col=1)
    fig.update_yaxes(title_text="Value (%)", title_font=dict(size=12), row=3, col=1)
    
    # Update x-axis labels
    fig.update_xaxes(title_text="Date", title_font=dict(size=12), row=3, col=1)
    
    # Add a watermark/annotation for clarity
    fig.add_annotation(
        text="A/D Line: Measures market breadth by tracking the cumulative difference between advancing and declining stocks",
        xref="paper", yref="paper",
        x=0.5, y=1.06,
        showarrow=False,
        font=dict(size=12, color="gray"),
        align="center"
    )
    
    return fig

def create_breadth_dashboard(ad_data, spy_data, merged_data):
    """
    Create a comprehensive market breadth dashboard
    
    Args:
        ad_data (pd.DataFrame): A/D line data
        spy_data (pd.DataFrame): SPY price data
        merged_data (pd.DataFrame): Merged data with divergence indicators
    
    Returns:
        plotly.graph_objects.Figure: Interactive Plotly dashboard
    """
    # Ensure data is aligned
    common_dates = ad_data.index.intersection(spy_data.index)
    
    # If no common dates, return empty figure with error message
    if len(common_dates) == 0:
        fig = go.Figure()
        fig.add_annotation(
            text="Error: No common dates between A/D line data and SPY data",
            xref="paper", yref="paper",
            x=0.5, y=0.5,
            showarrow=False,
            font=dict(size=20, color="red")
        )
        return fig
    
    # Filter data to common dates
    ad_aligned = ad_data.loc[common_dates]
    spy_aligned = spy_data.loc[common_dates]
    
    # Create subplot with 2x2 grid
    fig = make_subplots(rows=2, cols=2, 
                         specs=[[{"colspan": 2}, None],
                                [{"type": "xy"}, {"type": "xy"}]],
                         row_heights=[0.6, 0.4],
                         subplot_titles=("A/D Line vs SPY (Normalized)", 
                                         "Percentage of Advancing Stocks", 
                                         "Rolling Correlation"))
    
    # Normalize both series to start at 100
    # Find the first valid index for both datasets
    first_valid_idx = min(ad_aligned.index)
    
    # Plot A/D Line and SPY (normalized to 100 at start)
    ad_normalized = (ad_aligned['AD_Line'] / ad_aligned.loc[first_valid_idx, 'AD_Line']) * 100
    spy_normalized = (spy_aligned['Close'] / spy_aligned.loc[first_valid_idx, 'Close']) * 100
    
    fig.add_trace(
        go.Scatter(x=ad_aligned.index, y=ad_normalized, 
                   name="A/D Line (normalized)", 
                   line=dict(color='blue', width=2),
                   hovertemplate='Date: %{x}<br>A/D Line: %{y:.2f}<extra></extra>'),
        row=1, col=1
    )
    
    fig.add_trace(
        go.Scatter(x=spy_aligned.index, y=spy_normalized, 
                   name="SPY (normalized)", 
                   line=dict(color='green', width=2),
                   hovertemplate='Date: %{x}<br>SPY: %{y:.2f}<extra></extra>'),
        row=1, col=1
    )
    
    # Add a reference line at 100
    fig.add_hline(y=100, line_dash="dash", line_color="gray", row=1, col=1,
                 annotation_text="Starting Value (100)", annotation_position="bottom right")
    
    # Plot Percentage of Advancing Stocks with 10-day moving average
    fig.add_trace(
        go.Scatter(x=ad_aligned.index, y=ad_aligned['Pct_Advancing'], 
                   name="Daily % Advancing", 
                   line=dict(color='teal', width=1.5),
                   hovertemplate='Date: %{x}<br>% Advancing: %{y:.2f}%<extra></extra>'),
        row=2, col=1
    )
    
    fig.add_trace(
        go.Scatter(x=ad_aligned.index, y=ad_aligned['Pct_Advancing'].rolling(10).mean(), 
                   name="10-day MA", 
                   line=dict(color='darkblue', width=1.5, dash='dot'),
                   hovertemplate='Date: %{x}<br>10-day MA: %{y:.2f}%<extra></extra>'),
        row=2, col=1
    )
    
    # Add 50% reference line
    fig.add_hline(y=50, line_dash="dash", line_color="gray", row=2, col=1,
                 annotation_text="50% Advancing", annotation_position="bottom right")
    
    # Plot Rolling Correlation if merged data has it
    if 'Rolling_Corr' in merged_data.columns:
        # Ensure merged data is aligned with our timeframe
        merged_aligned = merged_data.loc[merged_data.index.intersection(common_dates)]
        
        fig.add_trace(
            go.Scatter(x=merged_aligned.index, y=merged_aligned['Rolling_Corr'], 
                       name="60-day Rolling Correlation", 
                       line=dict(color='purple', width=1.5),
                       hovertemplate='Date: %{x}<br>Correlation: %{y:.2f}<extra></extra>'),
            row=2, col=2
        )
        
        # Add reference lines for correlation
        fig.add_hline(y=0.7, line_dash="dash", line_color="green", row=2, col=2,
                     annotation_text="Strong Positive", annotation_position="bottom right")
        fig.add_hline(y=-0.7, line_dash="dash", line_color="red", row=2, col=2,
                     annotation_text="Strong Negative", annotation_position="bottom right")
        fig.add_hline(y=0, line_dash="dash", line_color="gray", row=2, col=2,
                     annotation_text="No Correlation", annotation_position="bottom right")
    
    # Update layout
    fig.update_layout(
        title={
            'text': "S&P 500 Market Breadth Dashboard",
            'font': {'size': 24, 'color': 'black'}
        },
        height=800,
        legend=dict(
            orientation="h", 
            yanchor="bottom", 
            y=1.02, 
            xanchor="center", 
            x=0.5,
            font=dict(size=10)
        ),
        hovermode="x unified",
        xaxis=dict(
            rangeselector=dict(
                buttons=list([
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=3, label="3m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all")
                ]),
                font=dict(size=10)
            )
        )
    )
    
    # Update y-axis labels
    fig.update_yaxes(title_text="Normalized Value (100 = Start)", title_font=dict(size=12), row=1, col=1)
    fig.update_yaxes(title_text="% of Stocks Advancing", title_font=dict(size=12), row=2, col=1)
    fig.update_yaxes(title_text="60-day Rolling Correlation", title_font=dict(size=12), row=2, col=2)
    
    # Update x-axis labels
    fig.update_xaxes(title_text="Date", title_font=dict(size=12), row=2, col=1)
    fig.update_xaxes(title_text="Date", title_font=dict(size=12), row=2, col=2)
    
    # Add a watermark/annotation for clarity
    fig.add_annotation(
        text="Normalized Comparison: Shows relative performance of A/D Line vs SPY starting from the same point (100)",
        xref="paper", yref="paper",
        x=0.5, y=1.06,
        showarrow=False,
        font=dict(size=12, color="gray"),
        align="center"
    )
    
    return fig

def save_plots_to_html(ad_line_plot, breadth_dashboard):
    """
    Save Plotly figures to HTML files for offline viewing
    
    Args:
        ad_line_plot (plotly.graph_objects.Figure): A/D Line analysis plot
        breadth_dashboard (plotly.graph_objects.Figure): Market breadth dashboard
    """
    try:
        ad_line_plot.write_html("ad_line_analysis.html")
        breadth_dashboard.write_html("market_breadth_dashboard.html")
        print("Plots saved as HTML files: ad_line_analysis.html and market_breadth_dashboard.html")
    except Exception as e:
        print(f"Error saving plots to HTML: {e}")

# Main execution
if __name__ == "__main__":
    # Get S&P 500 tickers
    print("Retrieving S&P 500 tickers...")
    sp500_tickers = get_sp500_tickers()
    print(f"Retrieved {len(sp500_tickers)} S&P 500 tickers.")
    
    # Set date range for analysis - using 1 year of data
    end_date = datetime.today().strftime('%Y-%m-%d')
    start_date = (datetime.today() - timedelta(days=365)).strftime('%Y-%m-%d')
    print(f"Analyzing data from {start_date} to {end_date}")
    
    # Get daily price data for S&P 500 stocks
    print("Downloading S&P 500 component data...")
    daily_data = get_daily_data(sp500_tickers, start_date, end_date)
    print(f"Data shape: {daily_data.shape}")
    
    # Get SPY data for comparison
    print("Downloading SPY data...")
    spy_data = get_spy_data(start_date, end_date)
    print(f"SPY data shape: {spy_data.shape}")
    
    # Calculate daily advances and declines
    print("Calculating daily advances and declines...")
    breadth_data = calculate_daily_advances_declines(daily_data)
    
    # Calculate A/D line and technical indicators
    print("Calculating A/D line and technical indicators...")
    ad_data = calculate_ad_line(breadth_data)
    print(f"A/D Line data shape: {ad_data.shape}")
    
    # Detect divergences
    print("Detecting divergences between A/D line and SPY...")
    merged_data = detect_divergences(ad_data, spy_data)
    print(f"Merged data shape: {merged_data.shape}")
    
    # Fit trendlines
    print("Fitting trendlines to A/D line...")
    trendlines = fit_trendlines(ad_data, 'AD_Line')
    print(f"Found {len(trendlines)} significant trendlines")
    
    # Create visualizations
    print("Creating visualizations...")
    ad_line_plot = create_ad_line_plot(ad_data, spy_data, merged_data, trendlines)
    breadth_dashboard = create_breadth_dashboard(ad_data, spy_data, merged_data)
    
    # Save visualizations to HTML files
    save_plots_to_html(ad_line_plot, breadth_dashboard)
    
    # Save the data
    daily_data.to_csv('sp500_daily_prices.csv')
    breadth_data.to_csv('sp500_breadth_data.csv')
    ad_data.to_csv('sp500_ad_line_data.csv')
    merged_data.to_csv('sp500_divergence_data.csv')
    
    # Display the plots
    ad_line_plot.show()
    breadth_dashboard.show()
    
    print("\nAnalysis complete! Key findings:")
    
    # Report on current market breadth conditions
    latest_date = ad_data.index[-1]
    latest_ad = ad_data.iloc[-1]
    
    print(f"\nLatest data as of {latest_date.strftime('%Y-%m-%d')}:")
    print(f"- A/D Line value: {latest_ad['AD_Line']:.2f}")
    print(f"- Percentage of advancing stocks: {latest_ad['Pct_Advancing']:.2f}%")
    
    # Check if A/D line is above/below moving averages
    sma50_status = "above" if latest_ad['AD_Line'] > latest_ad['AD_Line_SMA50'] else "below"
    sma200_status = "above" if latest_ad['AD_Line'] > latest_ad['AD_Line_SMA200'] else "below"
    
    print(f"- A/D Line is {sma50_status} its 50-day moving average")
    print(f"- A/D Line is {sma200_status} its 200-day moving average")
    
    # Check for recent divergences
    recent_divergences = merged_data['Divergence'].iloc[-30:].sum()
    print(f"- Number of divergence signals in last 30 days: {recent_divergences}")
    
    print("\nPlots have been saved as HTML files for interactive viewing.")

Retrieving S&P 500 tickers...
Retrieved 503 S&P 500 tickers.
Analyzing data from 2024-04-23 to 2025-04-23
Data shape: (250, 503)
SPY data columns after fixing: ['Close', 'High', 'Low', 'Open', 'Volume']
First few rows of SPY Close data: Date
2024-04-23    499.295227
2024-04-24    499.058228
2024-04-25    497.162354
2024-04-26    501.872437
2024-04-29    503.649811
Name: Close, dtype: float64
SPY data shape: (250, 5)
Calculating daily advances and declines...
Calculating A/D line and technical indicators...
A/D Line data shape: (250, 10)
Detecting divergences between A/D line and SPY...
Merged data shape: (249, 8)
Fitting trendlines to A/D line...
Found 4 significant trendlines
Creating visualizations...
Plots saved as HTML files: ad_line_analysis.html and market_breadth_dashboard.html



Analysis complete! Key findings:

Latest data as of 2025-04-22:
- A/D Line value: 5276.00
- Percentage of advancing stocks: 98.21%
- A/D Line is below its 50-day moving average
- A/D Line is above its 200-day moving average
- Number of divergence signals in last 30 days: 0

Plots have been saved as HTML files for interactive viewing.
